In [1]:
import datetime as dt
from pydantic import BaseModel, validator, root_validator
from typing import Optional
import json
import pytz

In [2]:
# datetime constructor with time zone information
# class datetime.datetime(year, month, day, hour=0, minute=0, second=0, microsecond=0, tzinfo=None, *, fold=0)

# tzinfo can be of class dt.timezone
# offset argument of dt.timezone has to be a dt.timedelta object
# name   argument of dt.timezone is optional
test_time = dt.datetime(2021, 2, 28, hour=16, minute=30, second=0, 
                        tzinfo=dt.timezone(offset=dt.timedelta(hours=1), name="Europe/Berlin"))

In [3]:
# convert to am/pm format
test_time.strftime("%I:%M %p")

'04:30 PM'

In [4]:
# create list of dates from to (including from and to)
from_t = dt.date(2021, 2, 1)
to_t = dt.date(2021, 2, 4)

In [5]:
# list comprehension
[from_t + dt.timedelta(days=i)  for i in range((to_t - from_t).days + 1)]

[datetime.date(2021, 2, 1),
 datetime.date(2021, 2, 2),
 datetime.date(2021, 2, 3),
 datetime.date(2021, 2, 4)]

## Solving the payload issue

In [21]:
class GoogleCalTime(BaseModel):
    """
    Can fill the "start" or "end" required field of a GoogleCalEvent!
    Has three fields: date, dateTime and timeZone, however, not all of them are required at a time.

    need date for all day events and dateTime for timed events
    timeZone: IANA timezone db name, NOT required if dateTime contains a time offset
    """

    date: Optional[dt.date] = None
    dateTime: Optional[dt.datetime] = None
    timeZone: Optional[str] = None

    # make sure that either date OR dateTime is provided (but not both at the same time)
    @root_validator(pre=True)
    def date_or_dateTime_provided(cls, values):
        date, datetime = values.get('date'), values.get('dateTime')
        assert (date is None and datetime is not None) or (date is not None and datetime is None), \
            "You have to provide a date for all day events OR a datetime for timed events!"
        return values

    # TODO: The next two TODO items are not required for this particular application, as astral will always return ...
    # TODO: ... datetime objects with timezone offset:
    # TODO: 1. check for valid IANA timezone names
    # TODO: 2. check that timezone is provided when dateTime does not contain a timezone offset


class GoogleCalEvent(BaseModel):
    """ The only required parameters of a google cal event are start and end. 'summary' is the correct
      field name of the event title. Can add additional fields later on when needed. """
    start: GoogleCalTime
    end: GoogleCalTime
    summary: str

    def payload(self):
        """pydantic provides method json() that serializes our model, especially datetime objects are converted
        to the isoformat sring automatically! For example, if a is an instance of GoogleCalEvent, we get sth like

        a.json() = '{"start": {"date": null, "dateTime": "2011-11-04T00:05:23+04:00", "timeZone": null},
                     "end": {"date": null, "dateTime": "2011-11-05T00:05:23+04:00", "timeZone": null},
                     "summary": "Calender event"}'

        This is almost what we want! We only have to create a dict from this json string and maybe we have to remove
        all fields containing null values (but maybe not!). This should be implemented in the payload method.
        """
        return json.loads(self.json())

In [7]:
start = GoogleCalTime(dateTime=test_time)
end = GoogleCalTime(dateTime=test_time + dt.timedelta(days=1))
event = GoogleCalEvent(summary="fake event", start=start, end=end)

In [8]:
print(event.json())

{"start": {"date": null, "dateTime": "2021-02-28T16:30:00+01:00", "timeZone": null}, "end": {"date": null, "dateTime": "2021-03-01T16:30:00+01:00", "timeZone": null}, "summary": "fake event"}


In [9]:
print(event.dict())

{'start': {'date': None, 'dateTime': datetime.datetime(2021, 2, 28, 16, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'Europe/Berlin')), 'timeZone': None}, 'end': {'date': None, 'dateTime': datetime.datetime(2021, 3, 1, 16, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'Europe/Berlin')), 'timeZone': None}, 'summary': 'fake event'}


In [10]:
event.dict()['start']['dateTime']

datetime.datetime(2021, 2, 28, 16, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'Europe/Berlin'))

In [11]:
event_dict=json.loads(event.json())

In [12]:
type(event_dict)

dict

In [13]:
event_dict

{'start': {'date': None,
  'dateTime': '2021-02-28T16:30:00+01:00',
  'timeZone': None},
 'end': {'date': None,
  'dateTime': '2021-03-01T16:30:00+01:00',
  'timeZone': None},
 'summary': 'fake event'}

In [14]:
event_dict['start']['date'] is None

True

### Another attempt at creating a timezone-aware datetime

In [15]:
naive_datetime = dt.datetime(2021, 2, 28, hour=16, minute=30, second=0)
timezone = pytz.timezone('Europe/Berlin')
aware_datetime =  timezone.localize(naive_datetime) 

In [16]:
aware_datetime

datetime.datetime(2021, 2, 28, 16, 30, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>)

In [22]:
def create_timezone_aware_datetime(year: int, month: int, day: int,
                                   hour: int, minute: int, second: int, timezone: str) -> dt.datetime:
    """Create timezone aware datetime object using the IANA timezone string, e.g.  'Europe/Berlin'."""

    naive_datetime = dt.datetime(year=year, month=month, day=day, hour=hour, minute=minute, second=second)
    timezone = pytz.timezone(timezone)
    aware_datetime = timezone.localize(naive_datetime)
    return aware_datetime

In [23]:
start = GoogleCalTime(dateTime=create_timezone_aware_datetime(year=2021, month=2, day=28, hour=16,
                                                                    minute=30, second=0, timezone='Europe/Berlin'))
end = GoogleCalTime(dateTime=create_timezone_aware_datetime(year=2021, month=2, day=28, hour=17, minute=30,
                                                                  second=0, timezone='Europe/Berlin'))
event = GoogleCalEvent(start=start, end=end, summary="test event")

payload = event.payload()

In [24]:
payload

{'start': {'date': None,
  'dateTime': '2021-02-28T16:30:00+01:00',
  'timeZone': None},
 'end': {'date': None,
  'dateTime': '2021-02-28T17:30:00+01:00',
  'timeZone': None},
 'summary': 'test event'}